In [1]:
! pip install -q pypdf2
! pip install -q chromadb
! pip install -q google.generativeai
! pip install -q langchain-google-genai
! pip install -q langchain
! pip install -q langchain_community
! pip install -q pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB

In [21]:
import sys
import os
import glob
import importlib
from typing import List, Optional
import chromadb

# import gemini_auth
# importlib.reload(gemini_auth)


from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma, DocArrayInMemorySearch
# from langchain.chains import RetrievalQA
from langchain import PromptTemplate

In [4]:
# Functions
def get_files(doc_folder):
    return glob.glob(f'{doc_folder}/*.pdf')

def split_docs(files):
    docs = []
    for file in files:
      # load pdf documents
      loader = PyPDFLoader(file)
      documents = loader.load()

      # split the documents into chunks
      text_splitter = RecursiveCharacterTextSplitter(chunk_size = CHUNK_SIZE, chunk_overlap = CHUNK_OVERLAP)
      docs.extend(text_splitter.split_documents(documents))

    return docs

def add_data_into_vector_db(docs, embedding_function):
    vectordb = Chroma.from_documents(
      docs, embedding_function,
      persist_directory=VECTOR_DIR
    )

In [24]:
os.environ['GOOGLE_API_KEY'] = 'your api key'

DOC_FOLDER = './doc'
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 150
VECTOR_DIR = './out/vector_dir'
add_data = False

llm = ChatGoogleGenerativeAI(model='gemini-pro')
embedding_func = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001"
)

question = 'what is the purpose of this document'

if __name__ == '__main__':
  files = get_files(DOC_FOLDER)
  docs = split_docs(files)
  if add_data:
    add_data_into_vector_db(
        docs, embedding_function=embedding_func
    )

  # to initialize the vector db for doc retrival
  # collect the chunks from vector db
  vectordb = Chroma(
      persist_directory=VECTOR_DIR,
      embedding_function=embedding_func
  )

  template = """
  context: {context}
  question: {question}
  """

  prompt = PromptTemplate(
      template=template,
      input_variables=['context', 'question']
  )

  llm_chain = prompt | llm

  question = 'who has posted this job?'

  context = []
  for doc in docs:
      context.append(doc.page_content)

  print(llm_chain.invoke({'context': context, 'question': question}))

content='The job has been posted by GULP.' response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'LOW', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]} id='run-1e7eeba2-35b1-4508-81c5-4f6a73afa725-0' usage_metadata={'input_tokens': 2184, 'output_tokens': 10, 'total_tokens': 2194}
